In [1]:
%pip install transformers datasets torch

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 484.9/484.9 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 50.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 43.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 49.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load the teacher model (large model)
teacher_model_name = "gpt2"
teacher_model = AutoModelForCausalLM.from_pretrained(teacher_model_name)

# Load the student model (smaller model, can use GPT-2 small as an example)
student_model_name = "distilgpt2"
student_model = AutoModelForCausalLM.from_pretrained(student_model_name)

# Load tokenizers
tokenizer = AutoTokenizer.from_pretrained(teacher_model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [3]:
import torch
import torch.nn.functional as F

def distillation_loss(student_logits, teacher_logits, temperature=2.0, alpha=0.5):
    """
    Compute distillation loss for the student model.

    Parameters:
    - student_logits: The student model's output logits
    - teacher_logits: The teacher model's output logits
    - temperature: The temperature for softening the teacher's logits
    - alpha: The weight of the distillation loss vs the hard target loss

    Returns:
    - The total distillation loss (combination of soft loss and hard loss)
    """
    # Soft targets for distillation
    soft_loss = F.kl_div(
        F.log_softmax(student_logits / temperature, dim=-1),
        F.softmax(teacher_logits / temperature, dim=-1),
        reduction='batchmean'
    ) * (temperature * temperature)

    # Hard targets (if available)
    # You can compute cross-entropy loss on the true labels if you have hard targets
    hard_loss = F.cross_entropy(student_logits, torch.argmax(teacher_logits, dim=-1))

    # Combined loss: weighted sum of soft and hard loss
    return alpha * soft_loss + (1 - alpha) * hard_loss


In [5]:
from torch.utils.data import DataLoader
from datasets import load_dataset
from transformers import AdamW
from transformers import AutoTokenizer

# Load a dataset (for example, the "wikitext" dataset)
dataset = load_dataset("wikitext", "wikitext-103-raw-v1")

# Create a DataLoader for training
train_data = dataset['train']
train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True)

# Optimizer
optimizer = AdamW(student_model.parameters(), lr=5e-5)

# Training loop
epochs = 3
temperature = 2.0
alpha = 0.5

tokenizer = AutoTokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token

for epoch in range(epochs):
    student_model.train()
    for batch in train_dataloader:
        # Tokenize input text
        inputs = tokenizer(batch['text'], return_tensors='pt', padding=True, truncation=True)

        labels = inputs.input_ids.clone()
        labels[labels == tokenizer.pad_token_id] = -100
        # Forward pass through teacher model
        with torch.no_grad():
            teacher_outputs = teacher_model(**inputs)
            teacher_logits = teacher_outputs.logits

        # Forward pass through student model
        student_outputs = student_model(input_ids=inputs.input_ids, labels=labels)

        # Get logits and loss from the output
        student_logits = student_outputs.logits
        loss = student_outputs.loss

        student_outputs = student_model(**inputs)
        student_logits = student_outputs.logits

        # Compute distillation loss
        loss = distillation_loss(student_logits, teacher_logits, temperature, alpha)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {loss.item()}")


RuntimeError: Expected target size [8, 50257], got [8, 213]